# Workshop 5
Hi and welcome to the fifth workshop. In this session we will take a look at a couple of the advanced stress packages because they tend to have input requirements that are somewhat uncommon to the regular stress packages. Specifically, we will focus on the lake package (LAK) and the multi-aquifer well package (MAW) to demonstrate how the different data requirements for these packages are developed and used in Flopy objects. We picked the MAW because of applicability and the LAK because it also introduces cell connectivity data input similar to the GWF-GWF package used for linking multiple models in MF6. Once you have a grasp of the input requirements for these packages the remaining advanced stress packages should not pose a significant obstacle. In addition to the advanced stress package creation, we will introduce some of the inbuilt post-processing capabilities of Flopy and provide applied use examples of how they can be used to good effect


In [ ]:
import os
import sys
import shutil
import platform
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy
from flopy.discretization import VertexGrid
from flopy.utils import Raster
from flopy.utils import GridIntersect
from flopy.utils.gridgen import Gridgen
gridgen_exe = "gridgen"
if platform.system() in "Windows":
    gridgen_exe += ".exe"
gridgen_exe = flopy.which("gridgen")
if gridgen_exe is None:
    msg = (
        "Warning, gridgen is not in your path. "
        "When you create the griden object you will need to "
        "provide a full path to the gridgen binary executable."
    )
    print(msg)
else:
    print("gridgen executable was found at: {}".format(gridgen_exe))

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Matplotlib version = {matplotlib.__version__}")

In [ ]:
ws5 = os.path.join('workshop_5') # here we are making a path not creating the folder
gis_f = os.path.join(ws5,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws5,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws5,'plots') # creating a sub-directory path for our plots
for path in [ws5,gis_f,model_f,plots_f]:
    if os.path.exists(path): # here we are asking if the path exists on the computer. 
        shutil.rmtree(path)# if it does exist, delete it and all the files in it
        os.mkdir(path) # then remake it
    else:
        os.mkdir(path) # if it doesn't exist then make the folder

In [ ]:
from helpers import ws5_model1
sim, gwf = ws5_model1(ws5,gis_f,model_f,plots_f)

In [ ]:
sim.run_simulation()

# A summary of the budget
Budget summaries should be checked consistently throughout the model development process as a rule. Recall that you configure the frequency of the summary printed to the list file via the OC package demonstrated in the previous workshop. Here we are going to use a convenient utility function of Flopy to load the list file and then convert the budget summaries into data frames that we can then use for plotting


In [ ]:
#Take a look at budget summary
lst = flopy.utils.Mf6ListBudget(os.path.join(model_f,"flow.lst")) # create a list file object for the current model run

df_flux, df_vol = lst.get_dataframes(start_datetime='31-12-2023') # use a method of the listfile object to convert the budget summaries to data frames

In [ ]:
df_flux

In [ ]:
df_vol

Staring at numbers may be tedious for some people who might prefer something visual. So lets create a summary plot of the budget fluxes. We first separate ins and outs using Pandas groupby dataframe method using axis=1, which means we are moving across the columns. What we pass into the groupby method is a lambda function. The lambda function is going to return the string trailing the final underscore in each column name. The groupby method can provide you with lots of information but we are only interested in the groups and their members so the final groups in the expression below will return a dictionary with the group as a key and the list of group members as the data.

In [ ]:
groups = df_flux.groupby(lambda x: x.split("_")[-1], axis=1).groups # splitting by the underscore to get in, out, or discrepancy groups.
groups

In [ ]:
df_flux_in = df_flux.loc[:, groups["IN"]] # create a dataframe view of the fluxes into the model
df_flux_in.columns = df_flux_in.columns.map(lambda x: x.split("_")[0]) # change the columns names here to exclude the "_IN"
# note use of the map method, which can be loosley translated to "apply to all"

In [ ]:
# repeat for outs
df_flux_out = df_flux.loc[:, groups["OUT"]]
df_flux_out.columns = df_flux_out.columns.map(lambda x: x.split("_")[0])

In [ ]:
# subtract the outs from the ins
df_flux_delta = df_flux_in - df_flux_out
# change from kL/d to ML/d
df_flux_delta = df_flux_delta*0.001

In [ ]:
# now plot a specific stress period using Pandas dataframe plot methods
spnum = 12
fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(1, 1, 1)
df_flux_delta.iloc[spnum, :].plot(kind="bar", grid=True,ax=ax) # note using stress period number here instead of date
plt.ylabel("ML/d")

Recharge and ET dominate the plot making a comparison of other fluxes difficult. You can solve this by dropping them from the plot

In [ ]:
fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(1, 1, 1)
df_flux_delta = df_flux_delta.drop(columns=["RCHA","EVTA"]) # note these are huge in comparison so we are dropping them for now.
df_flux_delta.iloc[spnum, :].plot(kind="bar", grid=True,ax=ax)
plt.ylabel("ML/d")

# Saving the heads for subsequent simulations
This was demonstrated but not used in the previous workshop. We will be running our model again so let’s save the initial condition heads for use again later


In [ ]:
headfile = os.path.join(model_f,"flow.hds")
hds = flopy.utils.binaryfile.HeadFile(headfile)
h = hds.get_data((0,0))
np.savetxt("iheads_array.txt",h[0]) # note this is not saved to the model folder because if I choose to rerun the script from scrtach it will be deleted.

# MAW package
Take a look at the MAW package input file in the MF6io.pdf document. You will notice that in addition to an options section you have to specify three other datasets. This includes package data, connection data and stress period data. In this example we will use a shapefile of bore locations to intersect the model grid and obtain the relevant nodes for each well. A function that we used to assign boundary conditions via intersections will be used. This function works well for lines points and polygons but will cause problems if you have multiple features


In [ ]:
import geopandas as gpd
def get_bnodes(shpfyl): # works with single and multiple line strings
    ix = GridIntersect(mg, method="vertex") # build our intersection object we will be applyitng this to a vertex grid only
    poly = gpd.read_file(shpfyl).geometry # read in the shapefile
    if len(poly)==1: # if the feature only has one entry
        return(ix.intersect(poly[0]).cellids) # note this is an array
    else: 
        ls = [] # if the feature has mutiple items i.e. points or multi-line strings
        for item in poly: # loop through the different geometries of each item
            nums = ix.intersect(item).cellids
            ls = [*ls,*nums]
        return(np.asarray(ls)) # # this will also be an array

In [ ]:
# adding in wells for explicit dewatering simulation
mg = gwf.modelgrid
wels = os.path.join(gis_f,"dewatering_wells.shp")
well_nodes = get_bnodes(wels)
wel_gdf = gpd.read_file(wels)
wel_gdf['node'] = well_nodes
wel_gdf.head()

Bore and pump depth are included in the shapefile so we will use this to configure our simulated bores. Note the MAW package offers many different options for well simulation behaviour and it is useful to know how you can best utilize the package for your specific simulation. However, in this instance we will adopt the 'THEIM' behaviour option. Each well has a single package data entry but can have multiple connection data entries depending on the different layers intercepted by the screen interval. We only have a three layer model and the wells may or may not reach layer 2. The algorithm below will use the layer elevation data and the bore drill depth to determine the number of connections needed.

In [ ]:
rad = 0.3 # assume all bores have a radius of 0.3 m
condeqn = "THEIM" # the string for setting the MAW behaviour option
pakdata = [] # initialze some empty lists for package data and connection data
condata = []
for i in range(len(wel_gdf)): # for each well
    welnum = i # assign a unique well number
    wel = wel_gdf.loc[i] # get the row entry for the well in the dataframe form the shapefile
    bottom = wel["Z"] - wel["Bore Depth"] # Work out the bottom elevation of the well
    count = 0 # set a count for the number of connections
    node = wel["node"] # this is the node in layer 1 that we got from our function
    start = mg.top[node] # set a starting elevation
    name = wel["Bore"] # get the well name
    for lay in range(3): # nlay =  3 Now loop thorugh each layers bottom elevation for the same node
        if mg.botm[lay][node]>bottom: # conditional on the layer elevation
            entry2 = (welnum,count,(lay,node),1,1,1,1)
            condata.append(entry2)
            count+=1
    entry = (welnum,rad,bottom,start,condeqn,count,name)
    pakdata.append(entry)

In [ ]:
pakdata

In [ ]:
condata

So it looks like some of the wells are only in the first layer while others are in the second layer. Once we have the package and connection data we can configure the stress period data. The rates needed from the shapefile are in the column named Dewateri_2. The "settings =" line is noteworthy. Each entry for single well is a list of lists, where each nested list has the well number as the first entry then a keyword followed by settings specific to that keyword. For example the keyword "status" requires another string as the setting, in this case "active". The keyword "rate_scaling" requires two float values (d and l). If you look in the MF6io.pdf document you can see what each keyword requires as its settings. Also very important, is making sure you comprehend how advanced stress packages turn on and off. This differs slightly from other stress packages.

In [ ]:
# Then finally we also need period data configured for each well
maw_pdata={} # creata a dictionary
pdata = [] # create a data list
for i in range(len(wel_gdf)): # loop thorugh each well
    wel = wel_gdf.loc[i] # Get the whole row of information for the well
    welnum = i # set the well number to be the loop counter *** Note this must be consistent with the numbering approach used previously
    r = -wel['Dewateri_2'] # assign the rate for the well
    d = wel["Z"] - wel["Pump Depth"] # get well max pumping depth *** note this was in mbgl so subtracting from well surface elevation
    l = 1.0 # setting the auto flow reducing window to 1.0 m above the max pump depth
    settings = [[welnum,"status","active"],[welnum,"rate",r],[welnum,"rate_scaling",d,l]] # creating the complete entry for a single well
    pdata = [*pdata,*settings]
maw_pdata[3] = pdata # all wells kick off 9 months prior to mining for this specific scenario which is stress period 4 in the model
maw_pdata

Now we can build our MAW package and an observation package as well. Note the way the observation file is created now. Because we have lots of different named boundaries - the well names in this case - we use a loop to create the entries for the observation package dictionary.

In [ ]:
# Now we can build our package
maw1 = flopy.mf6.ModflowGwfmaw(gwf,boundnames=True,
                               save_flows=True,
                               no_well_storage=True,
                               shutdown_kappa=0.01,
                               mfrcsv_filerecord="maw_reduce.csv",
                               nmawwells=len(well_nodes),
                               packagedata= pakdata, 
                               connectiondata= condata , 
                               perioddata = maw_pdata, 
                               filename="flow_1.maw",
                               pname="maw1")

# Setup obs entries for each well
ls =[]
for i in range(len(wel_gdf)):
    wel = wel_gdf.loc[i]
    name = wel["Bore"]
    ls.append((name,'maw',name)) # check the mF6io.pdf for other observations some advanced stress packages have many different types.

print(ls)

obs10_recarray = {
    "maw_obs.csv": ls
}
maw1.obs.initialize(
    filename="flow_1.maw.obs",
    digits=10,
    print_input=True,
    continuous=obs10_recarray,
)

In [ ]:
# using hdata from before
ihd_array=np.ones_like(mg.botm)
last_ssheads = np.loadtxt("iheads_array.txt")
ihd_array[:]=last_ssheads

ic = flopy.mf6.ModflowGwfic(
    gwf, pname="ic", strt=ihd_array, filename="flow.ic"
)

In [ ]:
sim.write_simulation()
sim.run_simulation()

# Visualising the dewatering simulation
The purpose of the wells in this situation was to dewater the upper aquifer completely before mining could begin. We are going to check this by using plots of cross-sections. Note there are certainly other ways to do this serves as a nice example to demonstrate some post-processing using cross-sections of the model. To begin with we are going to use a shell of the planned mine pit in Raster format. Lets take a quick look at it while mapping the elevation of the shell to an array called "spit-data", which we will use later.


In [ ]:
spit_shell = os.path.join(gis_f,'SouthPitShell.tif')
rio = Raster.load(spit_shell)
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
ax = rio.plot(ax=ax)
plt.colorbar(ax.images[0], aspect=30)
pmv = flopy.plot.PlotMapView(modelgrid=mg)
pmv.plot_grid(ax=ax, lw=0.3, color="black")
spit_data = rio.resample_to_grid(
    mg, band=rio.bands[0], method="nearest"
)
ax.set_xlim(rio.bounds[0],rio.bounds[1])
ax.set_ylim(rio.bounds[2],rio.bounds[3])

# Some plotting 
Here we will plot contours of hydraulic head and show the cross sections we will also be plotting. First we need the heads file from our model and then we will use the maximum and minimum values from that file to build a list of contour levels. We will use a mapview object and its built-in contour_array method to create the contour set. On top of that we will plot our cross-section line/s. For this we will use the mapview objects built in plot_shapefile feature. We also would like to indicate on the plot our cross section ends.


In [ ]:
# load the heads file
headfile = os.path.join(model_f,"flow.hds")
hds = flopy.utils.binaryfile.HeadFile(headfile)
# get the heads from the steady-state stress period
h0 = hds.get_data((0,0))
# get the heads from the final stress period- Note there is only one step in each stress period.
h = hds.get_data((0,12))
# get our contour levels using linspace
levels = np.linspace(np.min(h),np.max(h),20)
print(levels)

In [ ]:
# A water table cross section
#Lets plot a cross section of heads throough the model
# first we show where the cross section is
# Setup the figure
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1, aspect="equal")


mapview = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # create a mapview object for the current axis
pc = mapview.plot_array(h, cmap='viridis') # plot the heads array
colorbar = plt.colorbar(pc, aspect=30, shrink= 0.8) # create a colorbar
contour_set = mapview.contour_array(h, levels=levels, colors='w',linewidths = 0.75) # plot some contours using our levels
ax.clabel(contour_set, fmt='%.1f', colors='w', fontsize=8) # change the appearance of the contours


# Plot a shapefile of a cross-section line
shp = os.path.join(gis_f, "cross_sectionAB.shp") # path to the shapefile of a line I created in GIS
# plot the cross section line
patch_collection = mapview.plot_shapefile(shp, 
                                          radius=0, 
                                          lw=3, 
                                          edgecolor="red", 
                                          facecolor="None")
ax.axis('off') # turn off the axes

# position the labels for the cross section lines
lax,lay = patch_collection._paths[0].vertices[0] # depending on the line this may not be the correct vertex
lbx,lby = patch_collection._paths[0].vertices[1] # same again so be careful with this approach
text_kwargs = dict(fontsize = 16, color = 'k')
plt.text(lax,lay,'A',**text_kwargs,ha='right')
plt.text(lbx,lby,'B',**text_kwargs,ha='left')
plt.tight_layout()

In [ ]:
gwf.get_package_list

In [ ]:
drn5 = gwf.get_package('drn5')

In [ ]:
mg=gwf.modelgrid
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
pmv = flopy.plot.PlotMapView(modelgrid=mg, layer=0)
pmv.plot_grid(ax=ax, lw=0.3, color="black")
pmv.plot_bc(package=drn5,kper=12, color='grey') # Note these drains are asigned elevations above the model and were used here to highlight the pit only
pmv.plot_bc(package=maw1,color="green" )

# Plot a shapefile of a cross-section line
shp = os.path.join(gis_f, "cross_section_Spit.shp")
patch_collection1 = pmv.plot_shapefile(
    shp, radius=0, lw=3, edgecolor="red", facecolor="None"
)
shp2 = os.path.join(gis_f, "cross_section_Spit2.shp")
patch_collection2 = pmv.plot_shapefile(
    shp2, radius=0, lw=3, edgecolor="blue", facecolor="None"
)

# position the labels for the cross section lines
lax,lay = patch_collection1._paths[0].vertices[0] # depending on the line this may not be the correct vertex
lbx,lby = patch_collection1._paths[0].vertices[1] # same again so be careful with this approach
text_kwargs = dict(fontsize = 16, color = 'k')
plt.text(lax,lay,'A',**text_kwargs,ha='right')
plt.text(lbx,lby,'B',**text_kwargs,ha='left')

# position the labels for the cross section lines
lax2,lay2 = patch_collection2._paths[0].vertices[0] # depending on the line this may not be the correct vertex
lbx2,lby2 = patch_collection2._paths[0].vertices[1] # same again so be careful with this approach
text_kwargs = dict(fontsize = 16, color = 'k')
plt.text(lax2,lay2,'A',**text_kwargs,ha='right')
plt.text(lbx2,lby2,'B',**text_kwargs,ha='left')

# set the plotting bounds to only the area of interest
ax.set_xlim(rio.bounds[0]-250,rio.bounds[1]+250)
ax.set_ylim(rio.bounds[2]-250,rio.bounds[3]+250)
ax.set_title('South Pit Cross Sections')
ax.set_xlabel('Eastings')
ax.set_ylabel('Northings')
ax.ticklabel_format(style='plain') #  gets rid of the exponent offsets on the axis
plt.tight_layout()
fig.savefig(os.path.join(plots_f,'Pit_cross_sections.png'), dpi=300)

# USGSMap styling
The crosss-section plots we will create using Flopy's inbuilt styling method. The approach to plotting a cross-section was covered in a previous workshop so should be familiar to you.

In [ ]:
# import styles
from flopy.plot import styles

In [ ]:
# getting a line for the cross section. I'm sure there is an easier way to do this but it works
shp = os.path.join(gis_f, "cross_section_Spit.shp")
gdf = gpd.read_file(shp)
temp = gdf.geometry
x1 = temp.get_coordinates().iloc[0].x
y1 = temp.get_coordinates().iloc[0].y
x2 = temp.get_coordinates().iloc[1].x
y2 = temp.get_coordinates().iloc[1].y
line = {'line':[(x1,y1),(x2,y2)]}
print(x1,y1,x2,y2)

In [ ]:
with styles.USGSMap():
    fig, ax = plt.subplots(1, 1, figsize=(7, 3), dpi=300, tight_layout=True)
    extent = (0.0,690.0,320.0,420.0) # arrived at these through a bit of trial and error
    xsect = flopy.plot.PlotCrossSection(modelgrid=mg, ax=ax, line=line, extent=extent)
    # plot the surface and model grid
    xsect.plot_surface(h0[0],color = "green", lw = 1.0, ls=':', label='initial')
    wt1 = xsect.plot_surface(h[0], color="red", lw=1.0, label='9-month dewater')
    grd = xsect.plot_grid(lw=0.5)
    styles.xlabel(label="x-position (m)")
    styles.ylabel(label="elevation (m)")
    styles.heading(heading="Simulated watertable in Layer 1",fontsize=5)
    handles, labels = plt.gca().get_legend_handles_labels()
    labels, ids = np.unique(labels, return_index=True)
    handles = [handles[i] for i in ids]
    leg = styles.graph_legend(handles = handles, labels=labels, fontsize=5)
    styles.remove_edge_ticks(ax=ax)
    styles.graph_legend_title(leg=leg,title="")
    styles.xlabel(ax=ax,label='cross-section A to B position (m)',fontsize=5)
    styles.ylabel(ax=ax,label='cross-section elevation (m)',fontsize=5)
    ax.set_aspect(3.0)
plt.tight_layout()
fig.savefig(os.path.join(plots_f,'Dewater_cross_section.png'),dpi=300)

In [ ]:
shp = os.path.join(gis_f, "cross_section_Spit2.shp")
gdf = gpd.read_file(shp)
temp = gdf.geometry
x1 = temp.get_coordinates().iloc[0].x
y1 = temp.get_coordinates().iloc[0].y
x2 = temp.get_coordinates().iloc[1].x
y2 = temp.get_coordinates().iloc[1].y
line = {'line':[(x1,y1),(x2,y2)]}
print(x1,y1,x2,y2)

In [ ]:
with styles.USGSMap():
    fig, ax = plt.subplots(1, 1, figsize=(10, 3), dpi=300, tight_layout=True)
    extent2 = (0.0,1150.0,330.0,400.0)
    xsect2 = flopy.plot.PlotCrossSection(modelgrid=mg, ax=ax, line=line, extent=extent2)
    # plot the surface and model grid
    xsect2.plot_surface(h0[0],color = "green", lw = 1.0, ls=':', label='initial')
    wt1 = xsect2.plot_surface(h[0], color="blue", lw=1.0, label='9-month dewater')
    grd = xsect2.plot_grid(lw=0.5) 
    #plt.legend()
   # set labels using styles
    styles.xlabel(label="x-position (m)")
    styles.ylabel(label="elevation (m)")
    styles.heading(heading="Simulated watertable",fontsize=5)
    handles, labels = plt.gca().get_legend_handles_labels()
    labels, ids = np.unique(labels, return_index=True)
    handles = [handles[i] for i in ids]
    leg = styles.graph_legend(handles = handles, labels=labels, fontsize=5)
    styles.remove_edge_ticks(ax=ax)
    styles.graph_legend_title(leg=leg,title="")
    styles.xlabel(ax=ax,label='cross-section A to B position (m)',fontsize=5)
    styles.ylabel(ax=ax,label='cross-section elevation (m)',fontsize=5)
    ax.set_aspect(3.0)
plt.tight_layout()
fig.savefig(os.path.join(plots_f,'Dewater_cross_section2.png'),dpi=300)

# The Lake Package (LAK)
The lake package can have different configurations within a model. We will only demonstrate the explicit approach, the reasons for which will become obvious as we do it. The explicit method requires that you deactivate model cells that comprise the lake and then create connections to the lake from the cells adjacent to those that were deactivated. Cell connection information is available via the discretization packages. It should be clear that with a structured model grid this is relatively straight forward but could become very complex with an unstructured or vertices type grid. We're going to setup a lake in the mine pit that we plotted cross sections for. So the first step will be to figure out which cells are inside our lake void and then deactivate them via IDOMAIN. Note this is not as simple as using the polygon to select boundary cells because now we have to account for 3 dimensions.


In [ ]:
# So the idea here is to check the elevation of the cell node and if it is above the pit
# then it is inside the Lake and will have to be turned off.
# This means that we will need an array that has the pit elevations in it. 
# We did this earlier for 2D when we sampled the raster to the grid
spit_shell = os.path.join(gis_f,'SouthPitShell.tif')
rio = Raster.load(spit_shell)
spit_data = rio.resample_to_grid(
    mg, band=rio.bands[0], method="nearest"
)
# However, when mapping rasters smaller than the grid you can get errors. See the plot below.
# we should only see values different from -99999 where the ratser was, ... but we don't.
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
mapview = flopy.plot.PlotMapView(modelgrid=mg, ax=ax) # create a mapview object for the current axis
pc = mapview.plot_array(spit_data, cmap='viridis') # plot the heads array
ax.axis('off') # turn off the axes
# the implication here is that our array of elevations has errors outside of the pit area that need to be corrected

# Building an array filter
The imported model used a shapefile to create a drain package (drn5) for the mine pit. You saw this plotted previously. So we can get the nodes (think array indices) for layer 1 that are in the pit. We will then build an array the same shape as our modelgrid bottoms and assign the pit shell raster elevations to all cells that fall inside the pit area in all layers. Outside of the pit area we will assign values greater than our maximum surface elevation. We then use this array as a filter to compare our with our model cell centres. If a model cell centre is above (has a value greater than) our filter array then it must be inside the pit


In [ ]:
# first get our in the pit for layer 1
spit = os.path.join(gis_f,"spit_outer_poly.shp")
spit_nodes = get_bnodes(spit)

In [ ]:
# create a boolean array the size of mg.botm
mask = np.ones_like(spit_data,dtype="bool") # this effectivley translates to True at all locations in the 2D array
mask[spit_nodes.astype("int")] = 0 # Change locations inside the pit to False in the 2D array, we don't want these values affected
spit_data[mask]=1000.0 # change all values outside the pit to be 1000.0, which is above our model top elevation, only array indices flagged as true in the mask are affected

In [ ]:
# Using Matplotlib subplots to create a mutiple axes figure
fig, axs = plt.subplots(nrows = 1, # I want one row
                        ncols = 2, # I want three columns
                        figsize = (16,6),
                        subplot_kw={'aspect':'equal'},) # Each sub_plot must have aspect = equal to keep x and y ratios consistent

pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=axs[0]) #creating a mapview object and specifying the axes
pmv.plot_array(spit_data, cmap='viridis')
axs[0].axis('off') # turn off the axes 

pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=axs[1]) #creating a mapview object and specifying the axes
pmv.plot_array(spit_data, cmap='viridis')
axs[1].axis('off') # turn off the axes
axs[1].set_xlim(rio.bounds[0]-250,rio.bounds[1]+250)
axs[1].set_ylim(rio.bounds[2]-250,rio.bounds[3]+250)
plt.tight_layout()

In [ ]:
# Okay now we've fixed the errors so we can compare with cell centres, correct?
# Not quite. The cell centres are a 3D array and we only have a 2D for now so we need to fix that.
spit_data3D = np.vstack((spit_data,)*3) # Note how to create a repeating tuple, the comma is the key
print(np.shape(spit_data3D), np.shape(mg.zcellcenters)) # now we can do our comparison

In [ ]:
inpit = spit_data3D - mg.zcellcenters # anything with a negative value is in the pit void
# now we can make an Idomain array to turn off cells in the pit void
idom = np.zeros_like(mg.botm) # all deactivated
idom[inpit>0]=1 # activate model cells outside of pit void

fig, axs = plt.subplots(nrows = 1, # I want one row
                        ncols = 3, # I want three columns
                        sharey=True, # they can share the Y-axis to save space
                        subplot_kw={'aspect':'equal'}, # Each sub_plot must have aspect = equal to keep x and y ratios consistent
                        figsize=(12, 5)) # setting figure size Note had to increase the x dimension here

for i,ax in enumerate(axs):
    pmv = flopy.plot.PlotMapView(modelgrid=mg, layer = i, ax=ax) #creating a mapview object assigning a specific layer and specifying the axes
    pmv.plot_grid(ax=ax, lw=0.3, color="black") # plot the grid on the axes using linewidths = 0.3 and black lines
    pmv.plot_array(idom[i])
    ax.set_title(f'Model idomain Layer {i+1}') 
    ax.axis('off') # turn off the axes
    ax.set_xlim(rio.bounds[0]-250,rio.bounds[1]+250)
    ax.set_ylim(rio.bounds[2]-250,rio.bounds[3]+250)
plt.tight_layout()

# The connection data
This is the trickiest bit. However, if you get a feel for how to assign connections with the LAK package then you also know how to tackle nested models because the principle is the same. It may seem difficult but if you can grasp what information is stored in your modelgrid and how to access it then it can make your life much easier. To begin with we need to know what are the nodes adjacent to the inactive nodes. In a structured grid you could do a simple test because using unit increases or decreases in row and column to see if a neighbour of a deactivated cell is active thereby signalling a connection. With DISV and DISU we don't know the node numbers adjacent to our inactive cells, so how can we find teses ou?. Te followingscode blocks  serves as nt example for the benefits of a scripted modelling workflow.Unless the GUI you are using instead has a similar method for doing this you would have to set up all your connections by hand. Very painful.  Warning, this can be a bit of a brain drain so we'll take it slow.


In [ ]:
# now we need find to all active connections to the model grid from any deactivated cells
i,j = np.where(idom == 0) # will return the layer numbers as an array i, the node numbers as an array j
void_nodes = list(zip(i,j)) # zip i and j together to get the layer, node tuples for disv
void_nodes

In [ ]:
# Now for disv any cell that shares two or more vertices with the node is connected.
temp = {x[0]:list(zip(x[4:-1],x[5:])) for x in mg.cell2d} # creates vertex face pairs and links to nodes
hcon_ls, vcon_ls = [],[] # initialise lists for horizntal and vertical connections
for lay,node in void_nodes: # loop through all the deactivated nodes
    faces = temp[node] # get the vertes face pairs
    ls = [] # create an empty list to add active connections to
    # The list comprehension below loops through the dictionary we created checking if a pair of vertices is present in forwadr or backaward sequence
    ls = [key for key in temp.keys()
          for face in faces          
          if (face in temp[key] or (face[1],face[0]) in temp[key])]
    ls = [x for x in ls if idom[lay][x]==1] # ls is filtered to only contain node numbers for this layer if they are acctive, idomain=1
    if ls: # if there are any entries in ls
        ls=[(lay,x) for x in ls] # unpack the sub-list into the full list
        hcon_ls=[*hcon_ls,*ls] # create a list entry with the  the horizontal (layer,node) connections as a list
    if lay!=mg.nlay-1: # check if we are in the last layer
        if not (lay+1,node) in void_nodes: # if we are not in the last layer, check if an active vertical connection is present.
            vcon_ls.append((lay+1,node)) # create a list entry with the vertical connection (layer,node) in a list

In [ ]:
# For the connection data to the acive cells we need the following as a list of lists
# lakno = 1 in this case (user prescribed)
# iconn = loop counter (get from counter)
# cellid = cell the lake is connected to we get this from (node,layer) list
# claktype = vertical or horizontal
# bedleak = 'NONE' purely aquifer property control
# belev = cell bottom elevation (get from mg.bottoms)
# telev = assign as belev to internally set to top of cell
# connlen = we know this because of quadtree grid spacing but does depend on your grid 
# Connwidth = will be the same as connlen in this case but does depend on your grid 
width = 40
conlength = 40
lakno = 0
blk = 'NONE'
condata = []

for i,hcon in enumerate(hcon_ls):
    cell_bot = mg.botm[hcon[0],hcon[1]]
    condata.append([lakno,i,hcon,"HORIZONTAL",blk,cell_bot,cell_bot,conlength,width])
i+=1
for j,vcon in enumerate(vcon_ls):
    cell_bot = mg.botm[vcon[0],vcon[1]]
    condata.append([lakno,i+j,hcon,"VERTICAL",blk,cell_bot,cell_bot,conlength,width])
condata

In [ ]:
# Okay now lets add the Lake Package to the model
istage = 355.0 # approximate steady-state head elevation without lake
avg_rain = 6.8E-03 # approximatley 2500 mm/yr direct interception
evap = 0.004 # 4 mm/d or aprroximatley 1500 mm/yr
pakdata = [0,istage,len(condata),'Spit']
#tables = [0,'Spit_table.dat'] # You can provide a stage-volume-area lookup table if you need to

# Building a stress period dictionary as per normal
lak_spd={}
lak_spd[0] = [[0,"status","CONSTANT"],[0,"stage",istage]]
lak_spd[1] = [[0,"status","ACTIVE"],[0, "rainfall", avg_rain],[0,"evaporation",evap]]

lak = flopy.mf6.ModflowGwflak(
    gwf,
    boundnames=True,
    save_flows=True,
    stage_filerecord='Spit_lak_stage.out',
    budget_filerecord='Spit_budget.out',
    budgetcsv_filerecord='Spit_budget.csv',
    package_convergence_filerecord='Spit_convergence.csv',
    pname="Spit_lak",
    time_conversion=86400.00,
    length_conversion=1.0,
    mover=False,
    print_stage=False,
    nlakes=1,
    noutlets=0,
    #ntables=1, # if you have an SVA table for your lake/s then you turn this on
    packagedata=pakdata,
    connectiondata=condata,
    #tables=tables,
    outlets=None,
    perioddata=lak_spd,
)

obs_file = "flow.lak.obs"
csv_file = obs_file + ".csv"
obs_dict = {
    csv_file: [
        ('Spit_STG', "stage", (0,)),
        ('Spit_VOL', "volume", (0,)),
        ('Spit_SA', "surface-area", (0,)),
        ('Spit_WA', "wetted-area", 'Spit'),
        ('Spit_CND', "conductance", 'Spit'),
        ('Spit_RN', "rainfall", (0,)),
        ('Spit_EV', "evaporation", (0,)),
        ('Spit_AQ', "lak", 'Spit'),
    ]
}
lak.obs.initialize(
    filename=obs_file, digits=10, print_input=True, continuous=obs_dict
)

lak.write()
lak.obs.write()

# Ouch that was painful
Let’s run it for fun and have a look at the output table. But first we are going to setup the output control to save our budget to a csv that we can see for plotting later


In [ ]:
gwf.remove_package("MAW")
oc = flopy.mf6.ModflowGwfoc(gwf,budgetcsv_filerecord='mybudget.csv')

In [ ]:
sim.write_simulation()
sim.run_simulation()

# Alternative method for accessing budget data
In addition to loading your saved csv into a pandas dataframe you can also load the information directly with Flopy methods. Here we use the built in .output.budgetcsv() method to retrieve the LAK package list file budget entries as a csv. We will leverage the .output feature of Flopy more in subsequent sessions


In [ ]:
budcsv = gwf.output.budgetcsv()
budcsv.data

In [ ]:
# You can use the record array data-type information to list specific budget record array columns
budcsv.data['LAK(SPIT_LAK)_OUT']

# That's all folks
Next session will be on solute and variable density transport. See you then.